In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/transaction_dataset.csv')

In [3]:
df.head()

,Unnamed: 0,index,address,FLAG,avg_min_between_sent_tnx,avg_min_between_received_tnx,time_diff_between_first_and_last,sent_tnx,received_tnx,num_created_contracts,...,ERC20_min_val_sent,ERC20_max_val sent,ERC20_avg_val_sent,ERC20_min_val_sent_contract,ERC20_max_val_sent_contract,ERC20_avg_val_sent_contract,ERC20_uniq_sent_token_name,ERC20_uniq_rec_token_name,ERC20_most_sent_token_type,ERC20_most_rec_token_type
0,0,1,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8,0,844.26,1093.71,704785.63,721,89,0,...,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,1,2,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07,2958.44,1218216.73,94,8,0,...,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,2,3,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54,2434.02,516729.30,2,10,0,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,NaN,XENON
3,3,4,0x00038e6ba2fd5c09aedb96697c8d7b8fa6632e5e,0,10219.60,15785.09,397555.90,25,9,0,...,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,4,5,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61,10707.77,382472.42,4598,20,1,...,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS


In [4]:
## Addresses, index, and unnamed columns are not a useful feature to predict fraud - drop
df = df.drop(columns=['Unnamed: 0', 'index', 'address'])

In [5]:
df.head()

,FLAG,avg_min_between_sent_tnx,avg_min_between_received_tnx,time_diff_between_first_and_last,sent_tnx,received_tnx,num_created_contracts,unique_received_from_addresses,unique_sent_to_addresses,min_value_received,...,ERC20_min_val_sent,ERC20_max_val sent,ERC20_avg_val_sent,ERC20_min_val_sent_contract,ERC20_max_val_sent_contract,ERC20_avg_val_sent_contract,ERC20_uniq_sent_token_name,ERC20_uniq_rec_token_name,ERC20_most_sent_token_type,ERC20_most_rec_token_type
0,0,844.26,1093.71,704785.63,721,89,0,40,118,0.000000,...,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,0,12709.07,2958.44,1218216.73,94,8,0,5,14,0.000000,...,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,0,246194.54,2434.02,516729.30,2,10,0,10,2,0.113119,...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,NaN,XENON
3,0,10219.60,15785.09,397555.90,25,9,0,7,13,0.000000,...,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,0,36.61,10707.77,382472.42,4598,20,1,7,19,0.000000,...,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS


In [6]:
# Fill all NaN values with 0
df = df.fillna(0)

In [7]:
# Giving categorical features dummies
df['ERC20_most_sent_token_type'] = df['ERC20_most_sent_token_type'].astype(str)
df['ERC20_most_rec_token_type'] = df['ERC20_most_rec_token_type'].astype(str)

df['ERC20_most_sent_token_type'] = df['ERC20_most_sent_token_type'].replace(['0', '0.0'], 'None')
df['ERC20_most_rec_token_type'] = df['ERC20_most_rec_token_type'].replace(['0', '0.0'], 'None')

df = pd.get_dummies(df, columns=['ERC20_most_sent_token_type', 'ERC20_most_rec_token_type'], prefix = ['ERC20_most_sent_token', 'ERC20_most_rec_token'], dtype=int)

In [8]:
df.describe()

,FLAG,avg_min_between_sent_tnx,avg_min_between_received_tnx,time_diff_between_first_and_last,sent_tnx,received_tnx,num_created_contracts,unique_received_from_addresses,unique_sent_to_addresses,min_value_received,...,ERC20_most_rec_token_iDAG SPACE,ERC20_most_rec_token_iEx.ec Network Token,ERC20_most_rec_token_iXledger,ERC20_most_rec_token_minereum,ERC20_most_rec_token_savedroid,ERC20_most_rec_token_shellchains.com,ERC20_most_rec_token_timereum,ERC20_most_rec_token_vSlice,ERC20_most_rec_token_www.pnztrust.com,ERC20_most_rec_token_yocoinclassic
count,9841.000000,9841.000000,9841.000000,9.841000e+03,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,...,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000
mean,0.221421,5086.878721,8004.851184,2.183333e+05,115.931714,163.700945,3.729702,30.360939,25.840159,43.845153,...,0.000102,0.000102,0.000102,0.000508,0.000102,0.000305,0.000102,0.000102,0.003252,0.000102
std,0.415224,21486.549974,23081.714801,3.229379e+05,757.226361,940.836550,141.445583,298.621112,263.820410,325.929139,...,0.010080,0.010080,0.010080,0.022536,0.010080,0.017458,0.010080,0.010080,0.056934,0.010080
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,3.169300e+02,1.000000,1.000000,0.000000,1.000000,1.000000,0.001000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,17.340000,509.770000,4.663703e+04,3.000000,4.000000,0.000000,2.000000,2.000000,0.095856,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,565.470000,5480.390000,3.040710e+05,11.000000,27.000000,0.000000,5.000000,3.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,430287.670000,482175.490000,1.954861e+06,10000.000000,10000.000000,9995.000000,9999.000000,9287.000000,10000.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
## Save this new df to a new csv for the model
import os

csv_path = os.path.join(os.path.dirname(__file__), '..', 'data/' 'new_transaction_dataset_cleaned.csv' )
df.to_csv(csv_path, index=False)